<a href="https://colab.research.google.com/github/GeorgeDittmar/reddit-scan-datasets/blob/master/LM_Huggingface_finetune_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Finetuning GPT2 using HuggingFace and Tensorflow**

In this colab notebook we set up a simple outline of how you can use Huggingface to fine tune a gpt2 model on finance titles to generate new possible headlines. This notebook uses the hugginface finefuning scripts and then uses the TensorFlow version of the genreated models.

First begin setup by cloning transformers repo. We need to store the training script locally since there isnt an easier way to train tf based gpt2 models as far as I can see.

In [1]:
#Clone the transformers repo into the notebook
!git clone https://github.com/huggingface/transformers
!git clone https://github.com/GeorgeDittmar/reddit-scan-datasets.git

Cloning into 'transformers'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 57660 (delta 47), reused 59 (delta 33), pack-reused 57568
Receiving objects: 100% (57660/57660), 42.96 MiB | 5.55 MiB/s, done.
Resolving deltas: 100% (40454/40454), done.
Cloning into 'reddit-scan-datasets'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 141 (delta 6), reused 12 (delta 3), pack-reused 120
Receiving objects: 100% (141/141), 51.18 MiB | 5.98 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [2]:
# Clone should now be in the machine
!ls

reddit-scan-datasets  sample_data  transformers


Check to see what gpu we were granted. For Colab Pro it will vary between a Tesla V100 or P100. For normal colab it should be a k80

In [3]:
!nvidia-smi

Mon Dec 28 20:38:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Change directory location to be in the examples folder and then install any requirements

In [4]:
import os
os.chdir("transformers")
os.chdir("./examples/language-modeling")
!ls

README.md	  run_clm.py	   run_mlm.py	   run_plm.py
requirements.txt  run_mlm_flax.py  run_mlm_wwm.py


In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 163kB 14.1MB/s 
     |████████████████████████████████| 1.1MB 22.0MB/s 
     |████████████████████████████████| 245kB 49.6MB/s 
     |████████████████████████████████| 17.7MB 367kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [6]:
!ls

README.md	  run_clm.py	   run_mlm.py	   run_plm.py
requirements.txt  run_mlm_flax.py  run_mlm_wwm.py


In [7]:
!pip install pyarrow --upgrade

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (2.0.0)


In [8]:
!ls /content/reddit-scan-datasets/data_sets/investing/submission_titles_investing_100k.txt

/content/reddit-scan-datasets/data_sets/investing/submission_titles_investing_100k.txt


In [9]:
import os
os.chdir("/content/transformers/examples/")
os.chdir("./language-modeling")

In [10]:
# Need to install latest transformer packages from github so the scripts will run correctly
! pip install git+git://github.com/huggingface/transformers/

  Cloning git://github.com/huggingface/transformers/ to /tmp/pip-req-build-fii4adi2
  Running command git clone -q git://github.com/huggingface/transformers/ /tmp/pip-req-build-fii4adi2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 12.8MB/s 
     |████████████████████████████████| 2.9MB 31.9MB/s 
  Created wheel for transformers: filename=transformers-4.2.0.dev0-cp36-none-any.whl size=1527284 sha256=286ed09408e39f89cf6a92cbdc8de24217038dec6efc94593139f9befc937614
  Stored in directory: /tmp/pip-ephem-wheel-cache-__qy55tc/wheels/dc/e5/1e/3a2977a646558fca07585cadcf56aa4a910e995ba945961c4e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=684546f54ac2b31fd17b4109c1898cac578b4dbaef839e7a0af6d6a40840bade
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e

Set up data from a text file in the format <|title|> some data <|endoftext|> and split into training and eval sets.

In [13]:
"""
Now load the data line by line
"""
from sklearn.model_selection import train_test_split

with open('<path to text file>', 'r') as data:
  dataset = ["<|title|>" + x.strip() for x in data.readlines()]

train, eval = train_test_split(dataset, train_size=.9, random_state=2020)
print("training size:" + str(len(train)))
print("Evaluation size: " + str(len(eval))

with open('train_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))

with open('eval_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

training size:86114
Evaluation size: 9569


The script below will fine tune GPT2 on your text data that you setup above. This training step will take anywhre from tens of minutes to hours depending on how large your training set is, how many epochs you intend to train on, and if you are using colab or colab pro. We utilize mixed precision in this model to shave off some training time. For a large data set I was using for another experiment it saved us over 30 mins in training time.

In [14]:
!python run_clm.py \
--model_type gpt2-medium \
--model_name_or_path gpt2-medium \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--output_dir="<Path to your output dir>"

2020-12-28 20:41:44.222092: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
12/28/2020 20:41:46 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
12/28/2020 20:41:46 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=<Path to your output dir>, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=EvaluationStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Dec28_20-41-46_016c2c103498, logging_first_step=False, logging_steps=500, s

# **Using the model**
Next lets take our model we just trained and use it to generate some text! We will import the Tensorflow version of the gpt2 language model and set the from_pt flag to True. Then we load a pretrained tokenizer from huggingface. This may take some time to download the tokenizer data.

In [ ]:
# setup imports to use the model
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

model = TFGPT2LMHeadModel.from_pretrained("<path to model directory>", from_pt=True)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


Encoding sample text is now extremely simple using the pretrained tokenizer.

In [ ]:
input_ids = tokenizer.encode("Some text to encode", return_tensors='tf')

In [ ]:
# the tf tensor object
input_ids[0]

NameError: ignored

Next we will use the model to generate the text from our input sample. The parameters I used are based on trail and error from playing around with the huggingface tutorial, https://huggingface.co/blog/how-to-generate, which really goes into great detail on how to go about finding the best parameters for generating text. As well they dive into really good information on what each parameter does and how they play into one another.

In [ ]:
generated_text_samples = model.generate(
    input_ids, 
    max_length=150,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

In [ ]:
#Print output for each sequence generated above
for i, beam in enumerate(beam_output):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()

# **Conclusion**
And there you have it, a simple end to end outline on how you can use Colab, Huggingface, and Tensorflow to train and generate new text data using GPT-2. There is a lot of playing around with hyperparameters in the generate phase but given enough tweaking and time you can usually find something that works well with your data and task. I found that even with the larger GPT-2 model and more examples, it could still repeat itself a bit so something you have to generate a large number of sequences before you get a set that you like. Even OpenAI made note of this in their initial results for GPT-2 so if at first it doesnt generate what you want keep trying and playing with the parameters!

One tip I did notice was that if you do not setup your examples with a start token, then you run into the issue of repeated phrases more easily. Given more data that might be less of a problem but I ran into that a lot before putting in the start token of <|title|> in my exmaples. This start token also has the added benefit of giving you a generic starting point in the text generation so that each run is mostly unique from the last run if you do not care about having a specific prompt.